In [1]:
import pandas as pd
import os
import stats_distributions as stds
import statsmodels.stats.weightstats as stsm
import numpy as np
from numpy import sqrt, abs, round
from scipy.stats import norm

from scipy import stats


## One way ANOVA

In [58]:

os.chdir("C:\\Users\\satish\\Desktop")

df = pd.read_excel("MSE_data.xlsx", sheet_name = "Anova one way")
df.head(10)

,A,B,C,D
0,25,45,30,54
1,30,55,29,60
2,28,29,33,51
3,36,56,37,62
4,29,40,27,73


### H0: Means of all populations are equal
### H1: Means of atleast 1 population is not equal
### Right tail

In [59]:
# using scipy 
import scipy.stats as stats
# stats f_oneway functions takes the groups as input and returns ANOVA F and p value
f_stat, p_value = stats.f_oneway(df['A'], df['B'], df['C'], df['D'])
print(f_stat, p_value)

# Reject H0

17.492810457516338 2.639241146210922e-05


In [60]:
# change dataset structure for statsmodel
df_melt = pd.melt(df.reset_index(), id_vars=['index'], value_vars=['A', 'B', 'C', 'D'])
df_melt

,index,variable,value
0,0,A,25
1,1,A,30
2,2,A,28
3,3,A,36
4,4,A,29
5,0,B,45
6,1,B,55
7,2,B,29
8,3,B,56
9,4,B,40


In [64]:
# replace column names
df_melt.columns = ['index', 'treatments', 'value']
df_melt

,index,treatments,value
0,0,A,25
1,1,A,30
2,2,A,28
3,3,A,36
4,4,A,29
5,0,B,45
6,1,B,55
7,2,B,29
8,3,B,56
9,4,B,40


In [10]:
# use stats model
import statsmodels.api as sm
from statsmodels.formula.api import ols

# Ordinary Least Squares (OLS) model
model = ols('value ~ C(treatments)', data=df_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(treatments),3010.95,3.0,17.49281,0.000026
Residual,918.00,16.0,NaN,NaN


In [12]:
alpha = 0.05

print("test critical: %.5f ; alpha: %.5f" %(stds.pval_to_f(alpha, 3, 16, "right"), alpha))



test critical: 3.23887 ; alpha: 0.05000


In [ ]:
# F = 17.49 > test_critical => reject H0

In [14]:
# ANOVA table using bioinfokit v1.0.3 or later (it uses wrapper script for anova_lm)
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df=df_melt, res_var='value', anova_model='value ~ C(treatments)')
res.anova_summary
# Means are not all same

,df,sum_sq,mean_sq,F,PR(>F)
C(treatments),3.0,3010.95,1003.650,17.49281,0.000026
Residual,16.0,918.00,57.375,NaN,NaN


In [ ]:
# tretament = (smaple means - grand mean)
# residuals = (sample value - smaple mean)
# mean_sq = sum_sq/df 
# f = mstr/mse 
# p val using f and its df

## Tukey test for 1 way anova (Q distribution)

In [15]:
from bioinfokit.analys import stat
# perform multiple pairwise comparison (Tukey's HSD)
# unequal sample size data, tukey_hsd uses Tukey-Kramer test
res = stat()
res.tukey_hsd(df = df_melt, res_var = 'value', xfac_var = 'treatments', anova_model = 'value ~ C(treatments)')
res.tukey_summary

,group1,group2,Diff,Lower,Upper,q-value,p-value
0,A,B,15.4,1.692871,29.107129,4.546156,0.025070
1,A,C,1.6,-12.107129,15.307129,0.472328,0.900000
2,A,D,30.4,16.692871,44.107129,8.974231,0.001000
3,B,C,13.8,0.092871,27.507129,4.073828,0.048178
4,B,D,15.0,1.292871,28.707129,4.428074,0.029578
5,C,D,28.8,15.092871,42.507129,8.501903,0.001000


In [50]:
# We can see except for pair (A,C) all other pairs reject H0 and have significant diff.
df_melt['value'].describe()

count    20.000000
mean     41.450000
std      14.380085
min      25.000000
25%      29.000000
50%      36.500000
75%      54.250000
max      73.000000
Name: value, dtype: float64

## BONFERRONI CORRECTION method for 1 way anova

In [ ]:
# done to avoid error in stats library
df_melt['treatments'] = df_melt['treatments'].replace({'A':0, 'B':1, 'C':2, 'D':3})


In [66]:
import statsmodels.stats.multicomp as mc


comp = mc.MultiComparison(df_melt['value'], df_melt['treatments'] )
tbl, a1, a2 = comp.allpairtest(stats.ttest_ind, method= "bonf")



tbl

group1,group2,stat,pval,pval_corr,reject
0,1,-2.8918,0.0201,0.1209,False
0,2,-0.6375,0.5416,1.0,False
0,3,-7.2136,0.0001,0.0005,True
1,2,2.6015,0.0315,0.1893,False
1,3,-2.3837,0.0443,0.2658,False
2,3,-6.8767,0.0001,0.0008,True


In [ ]:
## We can see except for pair (A,C) and (C, D) all other pairs reject H0 and have significant diff.


## Two way anova

In [17]:

os.chdir("C:\\Users\\satish\\Desktop")

df = pd.read_excel("MSE_data.xlsx", sheet_name = "Anova two way repl")
df.head(10)

# 2 factors:
# Genotype - 6 levels
# Year - 3 levels

,Genotype,1_year,2_year,3_year
0,A,1.53,4.08,6.69
1,A,1.83,3.84,5.97
2,A,1.38,3.96,6.33
3,B,3.60,5.70,8.55
4,B,2.94,5.07,7.95
5,B,4.02,7.20,8.94
6,C,3.99,6.09,10.02
7,C,3.30,5.88,9.63
8,C,4.41,6.51,10.38
9,D,3.75,5.19,11.40


In [19]:
# reshape the d dataframe suitable for statsmodels package 
# you do not need to reshape if your data is already in stacked format. Compare d and d_melt tables for detail 
# understanding 
df_melt = pd.melt(df, id_vars=['Genotype'], value_vars=['1_year', '2_year', '3_year'])
# replace column names
df_melt.columns = ['Genotype', 'years', 'value']
df_melt.head()

# generate a boxplot to see the data distribution by genotypes and years. Using boxplot, we can easily detect the 
# differences between different groups
#sns.boxplot(x="Genotype", y="value", hue="years", data=d_melt, palette="Set3") 

,Genotype,years,value
0,A,1_year,1.53
1,A,1_year,1.83
2,A,1_year,1.38
3,B,1_year,3.60
4,B,1_year,2.94


In [20]:
# using statsmodel

import statsmodels.api as sm
from statsmodels.formula.api import ols
model = ols('value ~ C(Genotype) + C(years) + C(Genotype):C(years)', data = df_melt).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(Genotype),58.551733,5.0,32.748581,1.931655e-12
C(years),278.925633,2.0,390.014868,4.006243e-25
C(Genotype):C(years),17.122967,10.0,4.788525,2.230094e-04
Residual,12.873000,36.0,NaN,NaN


In [ ]:
# If you have unbalanced (unequal sample size for each group) data, you can perform similar steps as described for two-way 
# ANOVA with the balanced design but set `typ=3`. Type 3 sums of squares (SS) is recommended for an unbalanced design for 
# multifactorial ANOVA.

In [22]:
# using bioinfokit
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df = df_melt, res_var='value', anova_model='value~C(Genotype)+C(years)+C(Genotype):C(years)')
res.anova_summary

,df,sum_sq,mean_sq,F,PR(>F)
C(Genotype),5.0,58.551733,11.710347,32.748581,1.931655e-12
C(years),2.0,278.925633,139.462817,390.014868,4.006243e-25
C(Genotype):C(years),10.0,17.122967,1.712297,4.788525,2.230094e-04
Residual,36.0,12.873000,0.357583,NaN,NaN


In [ ]:
# The p value obtained from ANOVA analysis for genotype, years, and interaction are statistically significant (p<0.05).
# We conclude that type of genotype significantly affects the yield outcome, time (years) significantly affects the yield 
# outcome, and interaction of both genotype and time (years) significantly affects the yield outcome.

#from statsmodels.graphics.factorplots import interaction_plot
#import matplotlib.pyplot as plt
#fig = interaction_plot(x=d_melt['Genotype'], trace=d_melt['years'], response=d_melt['value'], 
#    colors=['#4c061d','#d17a22', '#b4c292'])
#plt.show()

#From the interaction plot, the interaction effect is significant between the Genotype and years because three lines are not 
# parallel (roughly parallel factor lines indicate no interaction - additive model). This interaction is also called ordinal 
# interaction as the lines do not cross each other.

## Tukey test for 2 way anova

In [27]:
# perform multiple pairwise comparison (Tukey HSD)
# unequal sample size data, tukey_hsd uses Tukey-Kramer test
res = stat()
# for main effect Genotype
res.tukey_hsd(df = df_melt, res_var = 'value', xfac_var = 'Genotype', 
              anova_model = 'value~C(Genotype)+C(years)+C(Genotype):C(years)')
res_df = res.tukey_summary
res_df

,group1,group2,Diff,Lower,Upper,q-value,p-value
0,A,B,2.040000,1.191912,2.888088,10.234409,0.001000
1,A,C,2.733333,1.885245,3.581421,13.712771,0.001000
2,A,D,2.560000,1.711912,3.408088,12.843180,0.001000
3,A,E,0.720000,-0.128088,1.568088,3.612145,0.135306
4,A,F,2.573333,1.725245,3.421421,12.910072,0.001000
5,B,C,0.693333,-0.154755,1.541421,3.478361,0.163609
6,B,D,0.520000,-0.328088,1.368088,2.608771,0.453066
7,B,E,1.320000,0.471912,2.168088,6.622265,0.001000
8,B,F,0.533333,-0.314755,1.381421,2.675663,0.425189
9,C,D,0.173333,-0.674755,1.021421,0.869590,0.900000


In [28]:
# filter significant diff
res_df[res_df['p-value'] > 0.05]

,group1,group2,Diff,Lower,Upper,q-value,p-value
3,A,E,0.720000,-0.128088,1.568088,3.612145,0.135306
5,B,C,0.693333,-0.154755,1.541421,3.478361,0.163609
6,B,D,0.520000,-0.328088,1.368088,2.608771,0.453066
8,B,F,0.533333,-0.314755,1.381421,2.675663,0.425189
9,C,D,0.173333,-0.674755,1.021421,0.869590,0.900000
11,C,F,0.160000,-0.688088,1.008088,0.802699,0.900000
13,D,F,0.013333,-0.834755,0.861421,0.066892,0.900000


In [29]:
# for main effect years
res.tukey_hsd(df = df_melt, res_var = 'value', xfac_var = 'years', 
              anova_model = 'value ~ C(Genotype) + C(years) + C(Genotype):C(years)')
res_df = res.tukey_summary
res_df

,group1,group2,Diff,Lower,Upper,q-value,p-value
0,1_year,2_year,2.146667,1.659513,2.633821,15.230432,0.001
1,1_year,3_year,5.521667,5.034513,6.008821,39.175794,0.001
2,2_year,3_year,3.375000,2.887846,3.862154,23.945361,0.001


In [31]:
# for interaction effect between genotype and years
res.tukey_hsd(df = df_melt, res_var = 'value', xfac_var = ['Genotype','years'], 
              anova_model = 'value ~ C(Genotype) + C(years) + C(Genotype):C(years)')
res_df = res.tukey_summary
res_df

,group1,group2,Diff,Lower,Upper,q-value,p-value
0,"(A, 1_year)","(A, 2_year)",2.38,0.548861,4.211139,6.893646,0.002439
1,"(A, 1_year)","(A, 3_year)",4.75,2.918861,6.581139,13.758326,0.001000
2,"(A, 1_year)","(B, 1_year)",1.94,0.108861,3.771139,5.619190,0.028673
3,"(A, 1_year)","(B, 2_year)",4.41,2.578861,6.241139,12.773520,0.001000
4,"(A, 1_year)","(B, 3_year)",6.90,5.068861,8.731139,19.985779,0.001000
...,...,...,...,...,...,...,...
148,"(E, 3_year)","(F, 2_year)",1.68,-0.151139,3.511139,4.866103,0.102966
149,"(E, 3_year)","(F, 3_year)",3.05,1.218861,4.881139,8.834294,0.001000
150,"(F, 1_year)","(F, 2_year)",0.74,-1.091139,2.571139,2.143402,0.900000
151,"(F, 1_year)","(F, 3_year)",5.47,3.638861,7.301139,15.843799,0.001000


In [32]:
# get significant diff
res_df[res_df['p-value'] > 0.05]

,group1,group2,Diff,Lower,Upper,q-value,p-value
11,"(A, 1_year)","(E, 1_year)",0.34,-1.491139,2.171139,0.984807,0.900000
18,"(A, 2_year)","(B, 1_year)",0.44,-1.391139,2.271139,1.274455,0.900000
21,"(A, 2_year)","(C, 1_year)",0.06,-1.771139,1.891139,0.173789,0.900000
24,"(A, 2_year)","(D, 1_year)",0.31,-1.521139,2.141139,0.897912,0.900000
25,"(A, 2_year)","(D, 2_year)",1.41,-0.421139,3.241139,4.084051,0.307915
28,"(A, 2_year)","(E, 2_year)",1.27,-0.561139,3.101139,3.678542,0.480455
30,"(A, 2_year)","(F, 1_year)",0.50,-1.331139,2.331139,1.448245,0.900000
31,"(A, 2_year)","(F, 2_year)",1.24,-0.591139,3.071139,3.591647,0.516386
34,"(A, 3_year)","(B, 2_year)",0.34,-1.491139,2.171139,0.984807,0.900000
37,"(A, 3_year)","(C, 2_year)",0.17,-1.661139,2.001139,0.492403,0.900000


## 2 way anova without replication

In [36]:
# Without replication is when we have no interaction among factors

df = pd.read_excel("MSE_data.xlsx", sheet_name = "Anova two way no repl")
df.head(10)

,Education,Gender,Score
0,BE,M,17
1,BE,F,13
2,MBA,M,16
3,MBA,F,18
4,PGDSA,M,18
5,PGDSA,F,15


In [37]:
# using statsmodel

import statsmodels.api as sm
from statsmodels.formula.api import ols

model = ols('Score ~ C(Education) + C(Gender)', data = df).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
anova_table

,sum_sq,df,F,PR(>F)
C(Education),4.333333,2.0,0.419355,0.704545
C(Gender),4.166667,1.0,0.806452,0.463944
Residual,10.333333,2.0,NaN,NaN


In [38]:
# using bioinfokit
from bioinfokit.analys import stat
res = stat()
res.anova_stat(df = df, res_var='Score', anova_model='Score ~ C(Education) + C(Gender)')
res.anova_summary

,df,sum_sq,mean_sq,F,PR(>F)
C(Education),2.0,4.333333,2.166667,0.419355,0.704545
C(Gender),1.0,4.166667,4.166667,0.806452,0.463944
Residual,2.0,10.333333,5.166667,NaN,NaN


In [ ]:
# Education and Gender do not affect Score